# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [69]:
def graph_identification():
    # YOUR CODE HERE
    
    final = []

    for G in P1_Graphs:
        degrees = dict(G.degree())
        degrees_value = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degrees_value]
    
        coeff = np.corrcoef(np.log(degrees_value), np.log(histogram))
        #print(abs(coeff)[0][1])
    
    
        #determine Prefertial Attachment by checking the linearity/correlation coeff
    
        if abs(coeff)[0][1] > 0.85:
            final.append('PA')
            continue
    
        #Small World with low probability of rewiring has low avg_shortest_path and high local_clustering
    
        avg_shortest_path = nx.average_shortest_path_length(G)
        #print(avg_shortest_path)
        local_clustering = nx.average_clustering(G)
        #print(local_clustering)
    
        if avg_shortest_path > 9 and local_clustering > 0.3:
            final.append('SW_L')
            
        else:
            final.append('SW_H')
    
    return final
    raise NotImplementedError()

In [70]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


In [72]:
ans_one

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [36]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier

    # YOUR CODE HERE
    #G.nodes(data=True)
    #df = pd.DataFrame(index = G.nodes())
    #df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    #df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    #create dataframe from nodes attribute of G

    d = dict(G.nodes(data = True))

    df = pd.DataFrame.from_dict(d).T

    #different measures

    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['triangle'] = pd.Series(nx.triangles(G))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))

    #split train and test data
    test = df[df['ManagementSalary'].isna()]

    train = df.copy()
    train = train.dropna()
    #train
    
    X_train = train.drop('ManagementSalary', axis =1)
    y_train = train['ManagementSalary']

    X_test = test.drop('ManagementSalary', axis =1)
    y_test = test['ManagementSalary']

    sc = StandardScaler()

    X_train_scaled = sc.fit_transform(X_train)
    #X_train_scaled
    #X_test_scaled

    X_test_scaled = sc.transform(X_test)

    #randomforestclassifier

    rfc = RandomForestClassifier()
    rfc.fit(X_train_scaled, y_train)

    y_pred = rfc.predict_proba(X_test_scaled)[:, 1]

    #return output
    #y_pred


    from sklearn.model_selection import GridSearchCV
    param_grid = {'criterion' :['entropy', 'gini'],
              'max_features': ['auto', 'sqrt','log2'],
              'min_samples_split': [2, 5, 10, 14]}
              

    clf = GridSearchCV(estimator = rfc, param_grid = param_grid, n_jobs = 1)


    clf.fit(X_train_scaled, y_train)

    #clf.best_params_

    #use best estimator to get more good result

    RFC = clf.best_estimator_

    RFC.fit(X_train_scaled, y_train)

    y_pred_new = RFC.predict_proba(X_test_scaled)[:, 1]
    final = pd.Series(y_pred_new, index = y_test.index)

    return final


    raise NotImplementedError()

In [37]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


In [73]:
ans_salary_preds

1      0.024872
65     0.873047
18     0.092512
215    0.790799
283    0.991111
         ...   
691    0.000000
788    0.024167
944    0.010000
798    0.000909
808    0.000000
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [31]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    
    G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))
    #print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")
    
    future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
    #future_connections.head(10)
    #list(nx.preferential_attachment(G, future_connections.index))[0][2]
    
    #value of preferential_attachments e[2] = PA in a tuple whereas e[0] = source and e[1] = target
    future_connections['preferential_attachment'] = [e[2] for e in nx.preferential_attachment(G, future_connections.index)] 
    future_connections['common_neigh'] = [len(list(nx.common_neighbors(G, e[0], e[1]))) for e in future_connections.index]
    #value of preferential_attachments e[2] = JC in a tuple whereas e[0] = source and e[1] = target
    future_connections['jaccard_coefficient'] = [e[2] for e in nx.jaccard_coefficient(G, future_connections.index)]
    #value of preferential_attachments e[2] = resource_allocation_index in a tuple whereas e[0] = source and e[1] = target
    future_connections['resource_allocation_index'] = [e[2] for e in nx.resource_allocation_index(G, future_connections.index)]

    #train test split

    train = future_connections.dropna()
    #train
    test = future_connections[future_connections['Future Connection'].isna()]
    #test

    X_train = train.copy()
    X_train = X_train.drop('Future Connection', axis =1)
    #X_train
    y_train = train['Future Connection']
    #y_train
    X_test = test.copy()
    X_test = X_test.drop('Future Connection', axis = 1)
    #X_test
    y_test = test['Future Connection']

    #Scaling

    sc = StandardScaler()
    X_train_scaled = sc.fit_transform(X_train)
    X_test_scaled = sc.transform(X_test)

    #model build

    rfc = RandomForestClassifier()
    rfc.fit(X_train_scaled, y_train)

    #prediction
    y_pred = rfc.predict_proba(X_test)[:, 1]

    final = pd.Series(y_pred, index = y_test.index)
    return final 

    # YOUR CODE HERE
    raise NotImplementedError()

In [32]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"


In [74]:
ans_prob_preds

(107, 348)    0.325
(542, 751)    0.110
(20, 426)     0.450
(50, 989)     0.110
(942, 986)    0.110
              ...  
(165, 923)    0.110
(673, 755)    0.060
(939, 940)    0.110
(555, 905)    0.110
(75, 101)     0.115
Length: 122112, dtype: float64